# Pagerank

These bits of code were run on a local machine to reduce latency while testing
to ensure that I had a properly functioning algorithm before running it on the
cloud.

## Imports

Importing a bunch of libraries and setting up the directory to read from.

In [1]:
import os
import networkx as nx
import numpy as np
from tqdm import tqdm
import re

directory_path = 'html/'

# read files in the directory
files = os.listdir(directory_path)

files = files

print(f'Found {len(files)} files')

Found 10000 files


## Outlinks

Generating an adjacency matrix consisting of outlinks from each page.
Saves to a file for later use.

In [2]:
# an adjacency matrix of outlinks
outlinks = np.zeros((len(files), len(files)))

print('Generating outlinks...')
print('This may take a while...')

for file in tqdm(files):
    # fetch the filename, which is the page title, which is everything before the .html
    filename = int(file.split('.')[0])

    # read each file in the directory
    with open(os.path.join(directory_path, file), 'r') as f:
        # use regex to parse for <a> tags with href; if a link is 
        # <a HREF='1000.html'>1000</a>, then the regex should return 1000
        regex = re.compile(r'<a HREF="(\d+).html">')

        # read the file
        html = f.read()

        # parse the html
        links = re.findall(regex, html)

        # for each link, add an edge from the current page to the linked page
        for link in links:
            outlinks[filename, int(link)] = 1

        # close the file
        f.close()

Generating outlinks...
This may take a while...


100%|██████████| 10000/10000 [00:03<00:00, 3137.49it/s]


## Inlinks

Generating an adjacency matrix consisting of inlinks to each page.
Saves to a file for later use.

In [3]:
inlinks = np.zeros((len(files), len(files)))

print('Generating inlinks...')
print('This may take a while...')

# generate inlinks from outlinks
inlinks = outlinks.transpose()

Generating inlinks...
This may take a while...


## Statistics

Uses the previously generated adjacency matrices to generate some statistics.

In [4]:
print('Outlinks')
print('---')
print(f'Average number of outlinks: {np.mean(np.sum(outlinks, axis=1))}')
print(f'Median number of outlinks: {np.median(np.sum(outlinks, axis=1))}')
print(f'Max number of outlinks: {np.max(np.sum(outlinks, axis=1))}')
print(f'Min number of outlinks: {np.min(np.sum(outlinks, axis=1))}')
print(f'Quintiles: {np.quantile(np.sum(outlinks, axis=1), [0.2, 0.4, 0.6, 0.8, 1])}')

print()

print('Inlinks')
print('---')
print(f'Average number of inlinks: {np.mean(np.sum(inlinks, axis=1))}')
print(f'Median number of inlinks: {np.median(np.sum(inlinks, axis=1))}')
print(f'Max number of inlinks: {np.max(np.sum(inlinks, axis=1))}')
print(f'Min number of inlinks: {np.min(np.sum(inlinks, axis=1))}')
print(f'Quintiles: {np.quantile(np.sum(inlinks, axis=1), [0.2, 0.4, 0.6, 0.8, 1])}')

Outlinks
---
Average number of outlinks: 122.6403
Median number of outlinks: 123.0
Max number of outlinks: 249.0
Min number of outlinks: 0.0
Quintiles: [ 49.  98. 147. 196. 249.]

Inlinks
---
Average number of inlinks: 122.6403
Median number of inlinks: 123.0
Max number of inlinks: 188.0
Min number of inlinks: 82.0
Quintiles: [113. 120. 125. 132. 188.]


## Pagerank Proper

The actual pagerank algorithm. Compared the bespoke pagerank implementation I
created to the one provided by networkx to ensure that they produced the same
results.

In [5]:
import networkx as nx

def pagerank(A, d=0.85, tol=0.005):
    num_nodes = A.shape[0]
    prev_pagerank = np.zeros(num_nodes) / num_nodes
    error = 1.0

    incoming_edges = [np.where(A[:, i] == 1)[0] for i in range(num_nodes)]
    outgoing_edges = [A[i].sum(axis=1) for i in incoming_edges]

    while error > tol:
        current_pagerank = prev_pagerank.copy()

        # calculate pagerank for each node
        current_pagerank = (1 - d) + d * np.array([np.sum(current_pagerank[incoming_edges[i]] / outgoing_edges[i]) for i in range(num_nodes)])

        error = np.linalg.norm(current_pagerank - prev_pagerank)
        prev_pagerank = current_pagerank

    # normalize by the sum of pageranks
    prev_pagerank /= np.sum(prev_pagerank)

    return prev_pagerank

print("Running bespoke pagerank...")
p = pagerank(outlinks)

# get the top 5 pages with highest page rank and their index
top_5 = sorted(range(len(p)), key=lambda i: p[i], reverse=True)[:5]
print("Top 5 pages with highest page rank:")
print("---")
for page in top_5:
    print(f"Page {page}: {p[page]}")

print()

# run page rank using networkx
print("Running networkx pagerank...")

G = nx.DiGraph(outlinks)
p = nx.pagerank(G, alpha=0.85)

# get top 5 pages with highest page rank
top_5 = sorted(p.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 pages with highest page rank:")
print("---")
for page in top_5:
    print(f"Page {page[0]}: {page[1]}")

Running bespoke pagerank...
Top 5 pages with highest page rank:
---
Page 2526: 0.00023818080090778595
Page 6846: 0.0002151123617518785
Page 5971: 0.00020964002178713986
Page 5778: 0.00020818389928806393
Page 1058: 0.00020662190926790968

Running networkx pagerank...
Top 5 pages with highest page rank:
---
Page 2526: 0.00023739511425470102
Page 6846: 0.0002152817763876744
Page 5971: 0.00020969301369874892
Page 5778: 0.00020816606740384702
Page 1058: 0.0002062924071280942
